### Compare to the results from Royden_Husson_06_gji

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.ThDSubduction0.RoydenHusson06 as RH06

#### Variables

In [ ]:
yr = 365.0 * 24.0 * 3600.0
cm_per_yr = 1e-2 / yr

# Vt = 1.0 * cm_per_yr
# Vm = 1.0 * cm_per_yr

Vt = 0.0 * cm_per_yr
Vm = 0.0 * cm_per_yr

Vr = 1.0 * cm_per_yr
L = 1000e3
a = L / 2.0
lbd = 700e3
mu = 1e20
n_hs = 100



Hele_Shaw = RH06.HELE_SHAW(Vt, Vm, Vr, L, lbd, mu, n_hs)

# As = np.zeros(n_hs)
As = 0.1 * np.ones(n_hs) * mu / lbd**2.0 * (Vt + Vm) # initial guess
Hele_Shaw.AssignA(As)

In [ ]:
# plot the result with the initial guess
x = 10e3 # 10 km
ym = L
fig, ax = plt.subplots()
Hele_Shaw.PlotVxdz(x, ym, ax=ax)

In [ ]:
J = Hele_Shaw.Jacobi_full(x, weight_inf=weight_inf)
# J = Hele_Shaw.Jacobi_full(x)

In [ ]:
x = 10e3 # 10 km
weight_inf = 10.0  # 0.0 means the infinite point is not included in the inversion
n_iter = 200
tolerance = 1e-6
# Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd(x)
Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd_full(x, weight_inf=weight_inf)
residuals = []

# Vx_dzs_nd_std = np.ones(n_hs)  # values on the surface of the slab
Vx_dzs_nd_std = np.ones(n_hs+1)  # values on the surface of the slab
Vx_dzs_nd_std[n_hs] = weight_inf * lbd * (Vt + Vm) / 2.0  # values to the infinity x
    
# residuals  
vx_dzs_diff = Vx_dzs_nd_std - Vx_dzs_nd
residual = np.linalg.norm(vx_dzs_diff, ord=np.inf)
residuals.append(residual)

for i in range(n_iter):

    if residual < tolerance:
        break

    # jacobian 
    J = Hele_Shaw.Jacobi_full(x, weight_inf=weight_inf)
    # J = Hele_Shaw.Jacobi_full(x)
    J1 = np.dot(J.transpose(), J)
    # J1 = np.dot(J, J.transpose())
    J1_inv = np.linalg.inv(J1)

    # apply iteration
    d_As = np.dot(J1_inv, np.dot(J.transpose(), vx_dzs_diff))
    As += d_As
    Hele_Shaw.AssignA(As)
    # Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd(x)
    Vx_dzs_nd = Hele_Shaw.Vxdz_array_nd_full(x, weight_inf=weight_inf)
    
    # residuals  
    vx_dzs_diff = Vx_dzs_nd_std - Vx_dzs_nd
    residual = np.linalg.norm(vx_dzs_diff, ord=np.inf)
    residuals.append(residual)

# export residual and history
print("Final residual: ", residual)

if residual < tolerance:
    # convergence
    end_i = i
else:
    # inconvergence
    end_i = i + 1
fig, ax = plt.subplots()
ax.semilogy(range(end_i + 1), residuals)

In [ ]:
# plot the result with the initial guess
x = 10e3 # 10 km
ym = L
fig, ax = plt.subplots()
Hele_Shaw.PlotVxdz(x, ym, ax=ax)
# ax.set_ylim([0.0, 2.0])